### Code to connect, acquire and store raw data from red pitaya from both CH1 and CH2 simulatenusly

### using config.json for manual device labelling

### data stored in data/[device]/[date]/session_000_CH#.csv

### author: Augusto

# 1. legacy data collection using SCPI server (sends buffers periodically)

In [7]:
import numpy as np
import pandas as pd
import redpitaya_scpi as scpi
import time
import datetime
import os
import json
import re
import sys

# Function to load configurations from config.json
def load_config(config_path='config.json'):
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Configuration file '{config_path}' not found.")
    
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    
    # Validate required keys
    required_keys = ["ip_address", "device_label", "decimation_factor", "base_dir", "session_delay_seconds"]
    for key in required_keys:
        if key not in config:
            raise KeyError(f"Missing required configuration key: '{key}'")
    
    return config

# Function to determine the next session index based on existing files
def get_next_session_index(date_dir):
    # Pattern to match session files e.g., session_023.csv
    pattern = re.compile(r'session_(\d{3})\.csv')
    existing_sessions = []
    
    if not os.path.exists(date_dir):
        return 1  # No existing sessions
    
    for filename in os.listdir(date_dir):
        match = pattern.match(filename)
        if match:
            session_num = int(match.group(1))
            existing_sessions.append(session_num)
    
    if not existing_sessions:
        return 1  # No existing sessions
    else:
        return max(existing_sessions) + 1  # Next session index

# Function to initialize CSV files with headers
def initialize_csv(filepath):
    with open(filepath, 'w') as f:
        f.write('voltage_ch1,voltage_ch2,timestamp\n')  # Header

# Function to append data to a single merged CSV
def append_to_csv(filepath, timestamp, voltage_ch1, voltage_ch2):
    # Create a list of timestamps: first element is timestamp, rest are empty strings
    timestamps = [timestamp] + [''] * (len(voltage_ch1) - 1)
    df = pd.DataFrame({
        'voltage_ch1': voltage_ch1,
        'voltage_ch2': voltage_ch2,
        'timestamp': timestamps,
    })
    df.to_csv(filepath, mode='a', header=False, index=False)

# Main data collection function
def collect_data(config):
    # Extract configurations
    IP = config['ip_address']
    device_label = config['device_label']
    decimation_factor = config['decimation_factor']
    base_dir = config['base_dir']
    session_delay = config['session_delay_seconds']
    
    # Initialize Red Pitaya connection
    try:
        rp_s = scpi.scpi(IP)
        print(f"Connected to Red Pitaya at {IP}")
    except Exception as e:
        print(f"Failed to connect to Red Pitaya at {IP}: {e}")
        sys.exit(1)
    
    # Create device directory if it doesn't exist
    device_dir = os.path.join(base_dir, device_label)
    os.makedirs(device_dir, exist_ok=True)
    
    # Get current date for organizing files
    current_date = datetime.datetime.now().strftime('%Y-%m-%d')
    date_dir = os.path.join(device_dir, current_date)
    os.makedirs(date_dir, exist_ok=True)
    
    # Determine the next session index
    session_index = get_next_session_index(date_dir)
    print(f"Starting at session index: {session_index}")
    
    try:
        while True:
            print(f"Starting data collection for session {session_index} with label '{device_label}'")
    
            # Get current timestamp for session start
            session_start_time = datetime.datetime.now().isoformat()

            # Configure Red Pitaya once before acquiring data
            try:
                rp_s.tx_txt('ACQ:RST')  # Reset acquisition
                rp_s.tx_txt(f'ACQ:DEC {decimation_factor}')  # Set decimation factor
                rp_s.tx_txt('ACQ:TRIG:LEV -1')  # Set trigger level
                rp_s.tx_txt('ACQ:START')  # Start acquisition
                print("Red Pitaya configured successfully.")
            except Exception as e:
                print(f"Error configuring Red Pitaya: {e}")
                sys.exit(1)
    
            # Trigger on both channels
            try:
                rp_s.tx_txt('ACQ:TRIG CH1_PE')
                rp_s.tx_txt('ACQ:TRIG CH2_PE')
                time.sleep(1)  # Wait for trigger and acquisition
            except Exception as e:
                print(f"Error triggering acquisition: {e}")
                time.sleep(session_delay)
                continue  # Skip to next iteration
    
            # Capture data from both channels
            try:
                rp_s.tx_txt('ACQ:SOUR1:DATA?')
                raw_data_ch1 = rp_s.rx_txt().strip('{}\n\r').split(',')
                signal_ch1 = np.array([float(x) for x in raw_data_ch1])
                signal_ch1 -= np.mean(signal_ch1)  # Remove DC component

                rp_s.tx_txt('ACQ:SOUR2:DATA?')
                raw_data_ch2 = rp_s.rx_txt().strip('{}\n\r').split(',')
                signal_ch2 = np.array([float(x) for x in raw_data_ch2])
                signal_ch2 -= np.mean(signal_ch2)  # Remove DC component

                if signal_ch1.size > 0 and signal_ch2.size > 0:
                    # Initialize file path for the current session
                    session_filepath = os.path.join(date_dir, f'session_{session_index:03d}.csv')
                    initialize_csv(session_filepath)

                    # Append data to the single CSV file
                    append_to_csv(session_filepath, session_start_time, signal_ch1, signal_ch2)
                    print(f"Session {session_index} data saved.")
                    session_index += 1  # Increment only after successful data saving
                else:
                    print(f"No data captured for session {session_index}.")
            except Exception as e:
                print(f"Error acquiring data: {e}")
    
            # Optional: Control data collection rate
            time.sleep(session_delay)  # Delay before the next session
    except KeyboardInterrupt:
        print("\nData collection stopped by user.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        print(f"Data collection terminated. Last session index: {session_index - 1}")

# Entry point of the script
if __name__ == "__main__":
    try:
        config = load_config('config.json')
    except (FileNotFoundError, KeyError) as e:
        print(f"Configuration Error: {e}")
        sys.exit(1)
    
    collect_data(config)

Connected to Red Pitaya at 192.168.8.214
Starting at session index: 185
Starting data collection for session 185 with label 'mac'
Red Pitaya configured successfully.
Session 185 data saved.
Starting data collection for session 186 with label 'mac'
Red Pitaya configured successfully.
Session 186 data saved.
Starting data collection for session 187 with label 'mac'
Red Pitaya configured successfully.
Session 187 data saved.
Starting data collection for session 188 with label 'mac'
Red Pitaya configured successfully.
Session 188 data saved.
Starting data collection for session 189 with label 'mac'
Red Pitaya configured successfully.
Session 189 data saved.
Starting data collection for session 190 with label 'mac'
Red Pitaya configured successfully.
Session 190 data saved.
Starting data collection for session 191 with label 'mac'
Red Pitaya configured successfully.
Session 191 data saved.
Starting data collection for session 192 with label 'mac'
Red Pitaya configured successfully.
Session 